In [1]:
import torch
import scanpy as sc
import numpy as np
from dataset import SingleCellRNACountsDataset
from dataprep import prep_sparse_data_for_training as prep_data_for_training
from encoder import EncodeZ, CompositeEncoder, EncodeNonZLatents
from decoder import Decoder
from train import run_training
from model import RemoveBackgroundPyroModel
import pyro
from pyro.infer import SVI, TraceEnum_ELBO, Trace_ELBO
import consts as consts
from pathlib import Path
import scipy.sparse as sp

In [17]:
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")

In [2]:
# to change here!
current_dir = globals()['_dh'][0]
# current_dir = Path(__file__).parent

In [3]:
model_type = "full"
fraction_empties = 0.5
expected_cell_count = 5_000
total_droplet_barcodes = 25_000
fpr = 0.01
low_count_threshold = 15

In [4]:
adata = sc.read_10x_h5(current_dir / ".." / "example_data" / "MS466" / "raw_feature_bc_matrix.h5")

/home/philipp/miniconda3/envs/torch_env2/lib/python3.9/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [5]:
data = {
    'matrix': sp.csr_matrix(adata.X),
    'barcodes':np.array(adata.obs_names, dtype=str),
    'gene_names': np.array(adata.var_names, dtype=str),
    'gene_ids': None,
    'feature_types': None
}

In [6]:
# here we also compute all the priors
dataset_obj = SingleCellRNACountsDataset(
    data=data,
    model_name=model_type,
    low_count_threshold=low_count_threshold,
    fpr=fpr,
    expected_cell_count=expected_cell_count,
    total_droplet_barcodes=total_droplet_barcodes, 
    fraction_empties=fraction_empties
)
dataset_obj

Trimming dataset for inference.
Including 36601 genes that have nonzero counts.


/home/philipp/Work/CellBenderMod/remove_background_mod/dataset.py:1366: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  empty_log_counts = mode(np.round(np.log1p(counts[counts > cut]),


Prior on counts in empty droplets is 180
Prior on counts for cells is 2763
Excluding barcodes with counts below 90
Using 5000 probable cell barcodes, plus an additional 20000 barcodes, and 49444 empty droplets.
Largest surely-empty droplet has 216.0 UMI counts.


In [7]:
dataset_obj.priors

{'n_cells': 5000,
 'cell_counts': 2763,
 'empty_counts': 180,
 'log_counts_crossover': 6.561465608076693,
 'd_std': 0.2725937153621734,
 'cell_prob': 0.1,
 'cell_logit': -2.197224577336219,
 'chi_ambient': tensor([1.7276e-07, 2.0595e-14, 2.0595e-14,  ..., 2.0595e-14, 5.7011e-06,
         5.1828e-07]),
 'chi_bar': tensor([1.7241e-08, 2.0552e-15, 2.0552e-15,  ..., 2.0552e-15, 2.6723e-06,
         5.5170e-07])}

In [8]:
count_matrix = dataset_obj.get_count_matrix()
# Configure pyro options (skip validations to improve speed).
pyro.enable_validation(False)
pyro.distributions.enable_validation(False)

In [9]:
# Load the dataset into DataLoaders.
frac = 1  # Fraction of barcodes to use for training
batch_size = int(min(300, frac * dataset_obj.analyzed_barcode_inds.size / 2))
batch_size

300

In [10]:
use_cuda = True
z_hidden_dims = [100]
d_hidden_dims = [10, 2]
p_hidden_dims = [100, 10]
z_dim = 10

In [11]:
fraction_empties = 0.5

In [12]:
learning_rate = 1e-4
epochs = 150

In [18]:
# Set up encode
encoder_z = EncodeZ(input_dim=count_matrix.shape[1],
                    hidden_dims=z_hidden_dims,
                    output_dim=z_dim,
                    input_transform='normalize')

encoder_other = EncodeNonZLatents(n_genes=count_matrix.shape[1],
                                    z_dim=z_dim,
                                    hidden_dims=consts.ENC_HIDDEN_DIMS,
                                    log_count_crossover=dataset_obj.priors['log_counts_crossover'],
                                    prior_log_cell_counts=np.log1p(dataset_obj.priors['cell_counts']),
                                    input_transform='normalize')

encoder = CompositeEncoder({'z': encoder_z,
                            'other': encoder_other})

AttributeError: 'CompositeEncoder' object has no attribute 'to'

In [14]:
test_X = torch.from_numpy(np.asarray(count_matrix.todense().astype(np.float32))).to(device)

In [15]:
encoder_z.forward(test_X)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

In [ ]:
# setu p decoder
decoder = Decoder(input_dim=z_dim,
                    hidden_dims=z_hidden_dims[::-1],
                    output_dim=count_matrix.shape[1])

In [ ]:
# set up the model
model = RemoveBackgroundPyroModel(model_type="full",
                                  encoder=encoder,
                                  decoder=decoder,
                                  dataset_obj=dataset_obj,
                                  use_cuda=use_cuda)
model

In [ ]:
# set up train and test loader
train_loader, test_loader = \
    prep_data_for_training(dataset=count_matrix,
                            empty_drop_dataset=dataset_obj.get_count_matrix_empties(),
                            random_state=dataset_obj.random,
                            batch_size=batch_size,
                            training_fraction=frac,
                            fraction_empties=fraction_empties,
                            shuffle=True,
                            use_cuda=use_cuda)

In [ ]:
# Set up the optimizer.
optimizer = pyro.optim.clipped_adam.ClippedAdam
optimizer_args = {'lr': learning_rate, 'clip_norm': 10.}
optimizer

In [ ]:
# Set up a learning rate scheduler.
minibatches_per_epoch = int(np.ceil(len(train_loader) / train_loader.batch_size).item())
scheduler_args = {'optimizer': optimizer,
                    'max_lr': learning_rate * 10,
                    'steps_per_epoch': minibatches_per_epoch,
                    'epochs': epochs,
                    'optim_args': optimizer_args}
scheduler = pyro.optim.OneCycleLR(scheduler_args)

In [ ]:
model.model_type

In [ ]:
if model.model_type == "simple":
    loss_function = Trace_ELBO()
else:
    loss_function = TraceEnum_ELBO(max_plate_nesting=1)
loss_function

In [ ]:
svi = SVI(model.model, model.guide, scheduler, loss=loss_function)
svi

In [ ]:
train_elbo, test_elbo, succeeded = run_training(model, svi, train_loader, test_loader,
                                                epochs=epochs, test_freq=5,
                                                final_elbo_fail_fraction=None,
                                                epoch_elbo_fail_fraction=None)